Implementacion del algoritmo Non-negative matrix factorization (NMF) en python para un sistema de recomendacion

Cargamos los datos y la transformamos a sparse matrix

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix,lil_matrix

RANDOM_STATE = 46
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

Observamos si existen usuarios o items que no esten en train pero si en test

In [ ]:
user_train_set = set(train['user'].unique())
user_test_set = set(test['user'].unique())
print(f"Numero de usuarios en train: {len(user_train_set)}")
print(f"Numero de usuarios en test: {len(user_test_set)}")
non_intersecting_users = user_test_set - user_train_set
print(f"Numero de usuarios en test que no estan en train: {len(non_intersecting_users)}")

item_train_set = set(train['item'].unique())
item_test_set = set(test['item'].unique())
print(f"Numero de items en train: {len(item_train_set)}")
print(f"Numero de items en test: {len(item_test_set)}")
non_intersecting_items = item_test_set - item_train_set
print(f"Numero de items en test que no estan en train: {len(non_intersecting_items)}")

Numero de usuarios en train: 73456
Numero de usuarios en test: 11426
Numero de usuarios en test que no estan en train: 4349
Numero de items en train: 171171
Numero de items en test: 25697
Numero de items en test que no estan en train: 14802


Podemos observar que en efecto existen 4349 usuarios y 14802 items que no estan en train, lo cual es un gran porcentaje de datos que el modelo no ha visto y va a tener que predecir.

Generamos una matriz sparse para el entrenamiento

In [3]:
# Find the number of users and items
num_users = max(train["user"].max(), test["user"].max()) + 1
num_items = max(train["item"].max(), test["item"].max()) + 1

# Convert to CSR sparse matrices
train_sparse = csr_matrix((train["rating"], (train["user"], train["item"])), shape=(num_users, num_items))

Definimos funciones de prediccion y error

In [9]:
def predict_value(user, item, W, H):
    """ Predict missing value for a given user-item pair """
    if user >= W.shape[0] or item >= H.shape[1]:
        print(f"Usuario con id {user} o item con id {item} no encontrado")
        return 0  # User or item not in training set

    predict_value = np.dot(W[user], H[:, item])
    return predict_value

def get_mse_error(sparse_data, W, H):
  mse_train_error = 0
  num_vals = 0
  rows, cols = sparse_data.nonzero()
  for row, col in zip(rows, cols):
    predicted_value = predict_value(row, col, W, H)
    mse_train_error += (sparse_data[row, col] - predicted_value) ** 2
    num_vals += 1

  mse_train_error /= num_vals
  return mse_train_error

Implementacion de sklearn de NMF

In [ ]:
import numpy as np
from sklearn.decomposition import NMF
num_components = 70
model = NMF(n_components=num_components, random_state=RANDOM_STATE,alpha_W = 0.001,alpha_H = 0.001,max_iter = 400,verbose = False)
W = model.fit_transform(train_sparse)
H = model.components_

print("Error de entrenamiento: ", get_mse_error(train_sparse, W, H))
print("Error de validacion: ", get_mse_error(val_sparse, W, H))

violation: 1.0
violation: 0.1591180526649301
violation: 0.10493736028066543
violation: 0.07548202220587953
violation: 0.055332187573304305
violation: 0.03965358883135004
violation: 0.02795975938731017
violation: 0.019980271920264664
violation: 0.01431331004288641
violation: 0.010582078937317852
violation: 0.008310907062502003
violation: 0.0071086153530074084
violation: 0.006372887975227188
violation: 0.0057360354537136626
violation: 0.0050158287423669205
violation: 0.004346889238173937
violation: 0.0038052606656561777
violation: 0.003354606218958939
violation: 0.0030016355004625515
violation: 0.00270903622349938
violation: 0.0024578466647988415
violation: 0.002277902983319306
violation: 0.002129690244748418
violation: 0.0019883597408573168
violation: 0.001878726900078003
violation: 0.001796086058026549
violation: 0.0017291201185336512
violation: 0.0016705613936294304
violation: 0.0016183829861253637
violation: 0.001581044830815781
violation: 0.0015263774457850796
violation: 0.001480284

Tras probar numerosas combinacion, sklearn no parece funcionar muy bien. De hecho la mayoria de valores que predice esta muy cercanos al 0. Una de las posibles razones puede ser el hecho de que trate los 0 como valores a predecir, en vez de valores nulos. Por ello vamos a realizar una implementacion manual.

Esta implementacion contara ademas con un factor de regularizacion para evitar el sobreajuste.

In [4]:
# Generamos nuestro propio NMF
from scipy.sparse import csr_matrix, coo_matrix

def train_custom_nmf(X, X_val, num_users,num_items, k=10,l2_reg = 0, max_iter=100, tol=1e-4, verbose=True):
    # Initialize W and H with small random values
    W = np.abs(np.random.rand(num_users, k))
    H = np.abs(np.random.rand(k, num_items))

    # Convert to COO format for faster iteration over non-zero elements
    X_coo = X.tocoo()

    # Indices and data of non-zero elements
    rows, cols, data = X_coo.row, X_coo.col, X_coo.data

    # Previous error to check for convergence
    prev_error = float('inf')

    for iter_num in range(max_iter):
        # Update W
        for i in range(num_users):
            # Get indices of non-zero elements in row i
            indices = X_coo.row == i
            if np.sum(indices) > 0:
                j_indices = cols[indices]
                x_values = data[indices]

                h_subset = H[:, j_indices]

                # Calculate numerator and denominator for the update rule
                numerator = np.zeros(k)
                denominator = np.zeros(k)

                for idx, (j, x_val) in enumerate(zip(j_indices, x_values)):
                    pred = np.dot(W[i], H[:, j])
                    if pred > 0:  # Avoid division by zero
                        numerator += x_val * H[:, j] / pred
                        denominator += H[:, j]

                # Regularization
                l2_term = 2 * l2_reg * W[i]
                denominator += l2_term

                # Update rule
                mask = denominator > 0
                W[i, mask] *= numerator[mask] / (denominator[mask] + 1e-10)

        # Update H
        for j in range(num_items):
            # Get indices of non-zero elements in column j
            indices = X_coo.col == j
            if np.sum(indices) > 0:
                i_indices = rows[indices]
                x_values = data[indices]

                # Skip if no non-zero values
                if len(x_values) == 0:
                    continue

                w_subset = W[i_indices, :]

                # Calculate numerator and denominator for the update rule
                numerator = np.zeros(k)
                denominator = np.zeros(k)

                for idx, (i, x_val) in enumerate(zip(i_indices, x_values)):
                    pred = np.dot(W[i], H[:, j])
                    if pred > 0:  # Avoid division by zero
                        numerator += x_val * W[i] / pred
                        denominator += W[i]

                # Regularization
                l2_term = 2 * l2_reg * H[:, j]
                denominator += l2_term

                # Update rule
                mask = denominator > 0
                H[mask, j] *= numerator[mask] / (denominator[mask] + 1e-10)


        # Calculate error only on non-zero elements
        error = get_mse_error(X, W, H)
        if X_val is None:
            val_error = 0
        else:
            val_error = get_mse_error(X_val, W, H)

        # Check for convergence
        improvement = prev_error - error
        if verbose:
            print(f"Iteration {iter_num+1}: Train error = {error:.6f}, Validation error = {val_error:.6f}, Improvement = {improvement:.6f}")

        if 0 <= improvement < tol:
            if verbose:
                print(f"Converged after {iter_num+1} iterations")
            break

        prev_error = error

    return W,H


Dividimos los datos en train y test, asegurandonos que train ha observado a los usuarios al menos una vez, este metodo no es perfecto (ya que es posible que haya items que train no haya visto), pero nos sirve.

In [5]:
import numpy as np
from scipy.sparse import lil_matrix

def train_test_split_sparse(ratings, test_percentage=0.2):
    # Get the indices of all non-zero elements
    nonzero_indices = ratings.nonzero()
    user_interactions = {}

    # Organize indices by user
    for i in range(len(nonzero_indices[0])):
        user = nonzero_indices[0][i]
        item = nonzero_indices[1][i]
        if user not in user_interactions:
            user_interactions[user] = []
        user_interactions[user].append(item)

    # Create binary masks
    train_mask = lil_matrix(ratings.shape, dtype=bool)
    test_mask = lil_matrix(ratings.shape, dtype=bool)

    # Split for each user
    for user, items in user_interactions.items():
        np.random.shuffle(items)
        split_idx = max(1, int(len(items) * (1 - test_percentage)))  # Ensure at least one interaction remains
        train_items, test_items = items[:split_idx], items[split_idx:]

        for item in train_items:
            train_mask[user, item] = True
        for item in test_items:
            test_mask[user, item] = True

    # Apply masks to get train and test sets
    train = ratings.multiply(train_mask)
    test = ratings.multiply(test_mask)

    return train, test

In [ ]:
train_sparse, val_sparse = train_test_split_sparse(train_sparse,test_percentage=0.1)

In [ ]:
W, H = train_custom_nmf(train_sparse,val_sparse, num_users,num_items, k=5 ,l2_reg = 0.001, max_iter=10, tol=1e-4, verbose=True)
train_error = get_mse_error(train_sparse, W, H)
print("Error de entrenamiento: ", train_error)

if val_sparse is not None:
  val_error = get_mse_error(val_sparse, W, H)
  print("Error de validacion: ", val_error)

else:
  val_error = train_error

Iteration 1: Train error = 1.745672, Validation error = 10.124449, Improvement = inf
Iteration 2: Train error = 1.103722, Validation error = 7.532552, Improvement = 0.641950
Iteration 3: Train error = 0.935341, Validation error = 7.163265, Improvement = 0.168382
Iteration 4: Train error = 0.839340, Validation error = 6.998991, Improvement = 0.096001
Iteration 5: Train error = 0.771041, Validation error = 6.898810, Improvement = 0.068298
Iteration 6: Train error = 0.718186, Validation error = 6.828712, Improvement = 0.052855
Iteration 7: Train error = 0.675421, Validation error = 6.776729, Improvement = 0.042765
Iteration 8: Train error = 0.639804, Validation error = 6.737272, Improvement = 0.035617
Iteration 9: Train error = 0.609505, Validation error = 6.707158, Improvement = 0.030299
Iteration 10: Train error = 0.583305, Validation error = 6.684325, Improvement = 0.026201
Error de entrenamiento:  0.5833048880318307
Error de validacion:  6.684324843112412


El dividir los datos en 2 conjuntos separados, parece que empeora los resultados, ya que se obtienen valores de validacion bastante bajos que son mmuy diferentes que el test de kaggle (valor de validacion de 6.7 vs 2.44 de test de kaggle).

Esto puede ser debido a la mala division de datos, o a la falta de datos de train. Por ello vamos a evitar utilizar el conjunto de validacion y se va a usar el test de kaggle como forma de validacion para el NMF.

In [7]:
train_sparse = csr_matrix((train["rating"], (train["user"], train["item"])), shape=(num_users, num_items))
val_sparse = None

Generamos un conjunto de hiperparametros a probar

In [ ]:
from itertools import product

param_grid = {
    'k': [5,10],
    'l2_reg': [0.001,0.1]
}


In [ ]:
from itertools import product

#No usamos validacion
val_sparse = None

# Generate all combinations
keys = param_grid.keys()
values = param_grid.values()

best_model = None
best_error = float('inf')
for combination in product(*values):
    param_combo = dict(zip(keys, combination))
    print(f"Training with parameters: {param_combo}")

    W, H = train_custom_nmf(train_sparse,val_sparse, num_users,num_items, k=param_combo['k'],l2_reg = param_combo['l2_reg'], max_iter=10, tol=1e-4, verbose=True)
    train_error = get_mse_error(train_sparse, W, H)
    print("Error de entrenamiento: ", train_error)

    if val_sparse is not None:
      val_error = get_mse_error(val_sparse, W, H)
      print("Error de validacion: ", val_error)

    else:
      val_error = train_error



    # Realizamos las predicciones de test y las guardamos
    submition = pd.DataFrame(columns=['ID', 'rating'])
    for index, row in test.iterrows():
        predicted_value = predict_value(row['user'], row['item'], W, H)
        new_row = pd.DataFrame([{'ID': row['ID'], 'rating': predicted_value}])
        submition = pd.concat([submition, new_row], ignore_index=True)

    submition.to_csv(f'./submission_NMF_{param_combo}.csv', index=False)

    # Guardamos el mejor modelo
    if val_error < best_error:
        best_error = val_error
        best_model = (W, H)

W = best_model[0]
H = best_model[1]

Training with parameters: {'k': 5, 'l2_reg': 0.001}
Iteration 1: Train error = 1.790799, Validation error = 0.000000, Improvement = inf
Iteration 2: Train error = 1.177148, Validation error = 0.000000, Improvement = 0.613651
Iteration 3: Train error = 1.008289, Validation error = 0.000000, Improvement = 0.168860
Iteration 4: Train error = 0.908586, Validation error = 0.000000, Improvement = 0.099702
Iteration 5: Train error = 0.836957, Validation error = 0.000000, Improvement = 0.071629
Iteration 6: Train error = 0.781400, Validation error = 0.000000, Improvement = 0.055558
Iteration 7: Train error = 0.736447, Validation error = 0.000000, Improvement = 0.044953
Iteration 8: Train error = 0.699027, Validation error = 0.000000, Improvement = 0.037419
Iteration 9: Train error = 0.667217, Validation error = 0.000000, Improvement = 0.031811
Iteration 10: Train error = 0.639723, Validation error = 0.000000, Improvement = 0.027494
Error de entrenamiento:  0.6397229506191231


<ipython-input-7-b03e252e988b>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submition = pd.concat([submition, new_row], ignore_index=True)


Training with parameters: {'k': 5, 'l2_reg': 0.1}
Iteration 1: Train error = 1.754693, Validation error = 0.000000, Improvement = inf
Iteration 2: Train error = 2.569946, Validation error = 0.000000, Improvement = -0.815252
Iteration 3: Train error = 1.892533, Validation error = 0.000000, Improvement = 0.677412
Iteration 4: Train error = 1.804586, Validation error = 0.000000, Improvement = 0.087947
Iteration 5: Train error = 1.648169, Validation error = 0.000000, Improvement = 0.156417
Iteration 6: Train error = 1.547540, Validation error = 0.000000, Improvement = 0.100629
Iteration 7: Train error = 1.461848, Validation error = 0.000000, Improvement = 0.085691
Iteration 8: Train error = 1.393262, Validation error = 0.000000, Improvement = 0.068586
Iteration 9: Train error = 1.336082, Validation error = 0.000000, Improvement = 0.057180
Iteration 10: Train error = 1.287981, Validation error = 0.000000, Improvement = 0.048101
Error de entrenamiento:  1.2879805450241542


<ipython-input-7-b03e252e988b>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submition = pd.concat([submition, new_row], ignore_index=True)


Training with parameters: {'k': 10, 'l2_reg': 0.001}
Iteration 1: Train error = 1.316765, Validation error = 0.000000, Improvement = inf
Iteration 2: Train error = 0.952668, Validation error = 0.000000, Improvement = 0.364097
Iteration 3: Train error = 0.827960, Validation error = 0.000000, Improvement = 0.124709
Iteration 4: Train error = 0.746358, Validation error = 0.000000, Improvement = 0.081602
Iteration 5: Train error = 0.684128, Validation error = 0.000000, Improvement = 0.062230
Iteration 6: Train error = 0.633846, Validation error = 0.000000, Improvement = 0.050282
Iteration 7: Train error = 0.591923, Validation error = 0.000000, Improvement = 0.041923
Iteration 8: Train error = 0.556229, Validation error = 0.000000, Improvement = 0.035694
Iteration 9: Train error = 0.525359, Validation error = 0.000000, Improvement = 0.030870
Iteration 10: Train error = 0.498327, Validation error = 0.000000, Improvement = 0.027032
Error de entrenamiento:  0.49832655462221886


<ipython-input-7-b03e252e988b>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submition = pd.concat([submition, new_row], ignore_index=True)


Training with parameters: {'k': 10, 'l2_reg': 0.1}
Iteration 1: Train error = 1.455678, Validation error = 0.000000, Improvement = inf
Iteration 2: Train error = 1.654947, Validation error = 0.000000, Improvement = -0.199270
Iteration 3: Train error = 1.453508, Validation error = 0.000000, Improvement = 0.201439
Iteration 4: Train error = 1.381044, Validation error = 0.000000, Improvement = 0.072464
Iteration 5: Train error = 1.302005, Validation error = 0.000000, Improvement = 0.079038
Iteration 6: Train error = 1.237705, Validation error = 0.000000, Improvement = 0.064300
Iteration 7: Train error = 1.181786, Validation error = 0.000000, Improvement = 0.055919
Iteration 8: Train error = 1.133223, Validation error = 0.000000, Improvement = 0.048563
Iteration 9: Train error = 1.090445, Validation error = 0.000000, Improvement = 0.042778
Iteration 10: Train error = 1.052411, Validation error = 0.000000, Improvement = 0.038035
Error de entrenamiento:  1.0524107560011104


<ipython-input-7-b03e252e988b>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submition = pd.concat([submition, new_row], ignore_index=True)


Training with parameters: {'k': 15, 'l2_reg': 0.001}


KeyboardInterrupt: 

| Hiperparametros  | Resultados en kaggle|
|------------|---------|
| {'k': 5, 'l2_reg': 0.001}     |  2.414  |
| {'k': 5, 'l2_reg': 0.1}     | 2.633 |
| {'k': 10, 'l2_reg': 0.001}     | 2.072  |
| {'k': 10, 'l2_reg': 0.1}     | 2.258 |


Los mejores parametros se obtienen con k = 10 y regularizacion muy pequeño de 0.001. Parece que incrementando k y disminuyendo la regularizacion, mejora bastante el algoritmo, probamos con mas iteraciones y k = 30

In [ ]:
W, H = train_custom_nmf(train_sparse,val_sparse, num_users,num_items, k=30 ,l2_reg = 0.001, max_iter=15, tol=1e-4, verbose=True)
train_error = get_mse_error(train_sparse, W, H)
print("Error de entrenamiento: ", train_error)

if val_sparse is not None:
  val_error = get_mse_error(val_sparse, W, H)
  print("Error de validacion: ", val_error)

else:
  val_error = train_error

Iteration 1: Train error = 1.017288, Validation error = 0.000000, Improvement = inf
Iteration 2: Train error = 0.780894, Validation error = 0.000000, Improvement = 0.236394
Iteration 3: Train error = 0.677252, Validation error = 0.000000, Improvement = 0.103642
Iteration 4: Train error = 0.603905, Validation error = 0.000000, Improvement = 0.073347
Iteration 5: Train error = 0.545884, Validation error = 0.000000, Improvement = 0.058021
Iteration 6: Train error = 0.497994, Validation error = 0.000000, Improvement = 0.047890
Iteration 7: Train error = 0.457529, Validation error = 0.000000, Improvement = 0.040464
Iteration 8: Train error = 0.422787, Validation error = 0.000000, Improvement = 0.034742
Iteration 9: Train error = 0.392587, Validation error = 0.000000, Improvement = 0.030200
Iteration 10: Train error = 0.366069, Validation error = 0.000000, Improvement = 0.026518
Iteration 11: Train error = 0.342586, Validation error = 0.000000, Improvement = 0.023483
Iteration 12: Train erro

Se obtiene un valor en kaggle de 1.526. Este valor mejora bastante en relacion al resto de implementaciones de NMF y seguramente pueda seguir mejorando, sin embargo no parece que se vaya a obtener la mejor solucion con esta implementacion, por tanto vamos a probar diferentes modelos.

Codigo para realizar las predicciones

In [ ]:
submition = pd.DataFrame(columns=['ID', 'rating'])
# Iterate row by row
for index, row in test.iterrows():
    predicted_value = predict_value(row['user'], row['item'], W, H)
    new_row = pd.DataFrame([{'ID': row['ID'], 'rating': predicted_value}])
    submition = pd.concat([submition, new_row], ignore_index=True)

submition.to_csv('./submission_NMF_def.csv', index=False)

<ipython-input-24-06eadbc14524>:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submition = pd.concat([submition, new_row], ignore_index=True)


Dado que en test puede haber items y usuarios que no se encuentren en entrenamiento, sustituimos esos valores por la media.

In [11]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
user_train_set = set(train['user'].unique())
item_train_set = set(train['item'].unique())

submition = pd.DataFrame(columns=['ID', 'rating'])
# Iterate row by row
for index, row in test.iterrows():
    if row['user'] not in user_train_set and row['item'] not in item_train_set:
        predicted_value = train['rating'].mean()
    elif row['user'] not in user_train_set:
        predicted_value = train[train['item'] == row['item']]['rating'].mean()
    elif row['item'] not in item_train_set:
        predicted_value = train[train['user'] == row['user']]['rating'].mean()
    else:
        predicted_value = predict_value(row['user'], row['item'], W, H)

    new_row = pd.DataFrame([{'ID': row['ID'], 'rating': predicted_value}])
    submition = pd.concat([submition, new_row], ignore_index=True)

submition.to_csv('./submission_NMF_def.csv', index=False)

<ipython-input-11-1218c8665c7c>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submition = pd.concat([submition, new_row], ignore_index=True)


Este metodo de reemplaza valores no vistos por la media mejora los resultados. Esto es debido a que el es muy dificil que el modelo pueda predecir datos de usuarios o items que nunca ha visto. Pasando de obtener un valor de 1.526 a 1.399.